In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import reuters
import pickle

In [2]:
with open('vocab.txt') as vocab_file:
    lines = vocab_file.readlines()
vocab = [line.strip() for line in lines]

testdata = pd.read_table('testdata.txt', header=None)
n = testdata.shape[0]

with open('dataFile','rb') as fr:
    un_freq = pickle.load(fr)
    bi_freq = pickle.load(fr)

In [3]:
V = len(un_freq)

letters = "abcdefghijklmnopqrstuvwxyz"
letters_upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
other = " -'"

In [11]:
def distance_1(word):
    total = set()
    for i in range(0,len(word)):
        l = word[:i]
        r = word[i:]
        alphabet = letters
        if len(word) > 1 and i > 0:
            total.update(set([l[:i-1]+r[0]+l[i-1]+r[1:]])) #Transposition
            if i < len(word):
                alphabet = letters+other # Punctuation cases
        total.update(set([l+c+r for c in alphabet]+[l+r+c for c in alphabet])) #Insertion
        total.update(set([l+r[1:]])) #Deletion
        total.update(set([l+c+r[1:] for c in alphabet])) #Substitution
    return total

In [16]:
def generate_candidates(sentence):
    candidates = list()
    for p,word in enumerate(sentence):
        for replace in distance_1(word):
            if replace in vocab:
                candidate = sentence[:p]+[replace]+sentence[p+1:]
                if candidate not in candidates:
                    candidates.append(candidate)
    return candidates

In [17]:
generate_candidates(word_tokenize("a quick brown fox"))

[['ax', 'quick', 'brown', 'fox'],
 ['ah', 'quick', 'brown', 'fox'],
 ['h', 'quick', 'brown', 'fox'],
 ['k', 'quick', 'brown', 'fox'],
 ['al', 'quick', 'brown', 'fox'],
 ['am', 'quick', 'brown', 'fox'],
 ['ha', 'quick', 'brown', 'fox'],
 ['as', 'quick', 'brown', 'fox'],
 ['a', 'quick', 'brown', 'fox'],
 ['u', 'quick', 'brown', 'fox'],
 ['x', 'quick', 'brown', 'fox'],
 ['at', 'quick', 'brown', 'fox'],
 ['f', 'quick', 'brown', 'fox'],
 ['na', 'quick', 'brown', 'fox'],
 ['l', 'quick', 'brown', 'fox'],
 ['r', 'quick', 'brown', 'fox'],
 ['b', 'quick', 'brown', 'fox'],
 ['g', 'quick', 'brown', 'fox'],
 ['ca', 'quick', 'brown', 'fox'],
 ['ab', 'quick', 'brown', 'fox'],
 ['ad', 'quick', 'brown', 'fox'],
 ['p', 'quick', 'brown', 'fox'],
 ['pa', 'quick', 'brown', 'fox'],
 ['e', 'quick', 'brown', 'fox'],
 ['n', 'quick', 'brown', 'fox'],
 ['w', 'quick', 'brown', 'fox'],
 ['ya', 'quick', 'brown', 'fox'],
 ['la', 'quick', 'brown', 'fox'],
 ['ta', 'quick', 'brown', 'fox'],
 ['an', 'quick', 'brown', 'f

In [19]:
N = len(reuters.words())

In [20]:
N

1720901

In [21]:
reuters_list = reuters.words()

In [22]:
N= len(reuters_list)

In [23]:
with open('dataFile','rb') as fr:
    un_freq = pickle.load(fr)
    bi_freq = pickle.load(fr)
    V = pickle.load(fr)
    N = pickle.load(fr)

In [24]:
N

1720901

In [25]:
V

41600

In [26]:
def log_smoothed_prob(pre,next):
    prob = math.log10((bi_freq[(pre,next)]+1))-math.log10((un_freq[pre]+V))
    return prob

In [40]:

def sentence_prob(sentence):
    log_prob = 0
    for p,word in enumerate(sentence):
        if p == 0:
            log_prob = math.log10(un_freq[word]+1) - math.log10(N+V)
        else:
            log_prob = log_prob + log_smoothed_prob(sentence[p-1],word)
    return log_prob

In [41]:
sentence_prob(word_tokenize("a quick brown fox"))

-8.72688813250023

In [42]:
sentence_prob(word_tokenize("a quick brow fox"))

-8.72683593678093

In [35]:
sentence_prob(word_tokenize("a quick bro fox"))

-14.972965308852515

In [38]:
sentence_prob(word_tokenize("asfox"))

-6.246129372071583

In [39]:
sentence_prob(word_tokenize("sdadfx"))

-6.246129372071583

In [43]:
un_freq["word"]

13

In [44]:
un_freq["words"]

14

In [47]:
bi_freq[('a','quick')]

8

In [48]:
un_freq['a']

23492

In [49]:
bi_freq[('brown','fox')]

0

In [50]:
bi_freq[('a','quack')]

0